# Global Imports


In [ ]:
import mysql.connector
import random
import pandas as pd
import numpy as np
from itertools import combinations, product
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
import warnings
import matplotlib.gridspec as gridspec

sns.set(style="ticks")

warnings.filterwarnings("ignore")
import math
import collections

import bisect

import ast
import datetime
import itertools
import json
from functools import partial
import os

In [ ]:

from src.data import Content, Titles, DB, Meta
from src.filtering import SceneHeuristic
from src.utils import Cluster, CorrelationStructure, CStruct, Scenes, User, Users
from src.scoring import Scores
from src.models import K_Mean


# Clustering

*Theory and Steps*
https://docs.google.com/document/d/1Gis0vZxjqM5tR-G0LgE7NwJ02zgRdmys1sNmVLqSirc/edit?usp=sharing

*Results*
https://docs.google.com/document/d/1WuITMTViVCM6BXpMFqX_l8C_9Lgr6zZwLdZ_o-M_jos/edit#heading=h.cuzpdgemp057


### Retrieval




In [ ]:


# read only web DB
connector = {"host": os.environ.get('host'),
              "user":os.environ.get('user'),
              "password":os.environ.get('password')}

titles = Titles(connector, 'movie', topScenes=6)



In [ ]:
db = DB(connector['host'],connector['user'],connector['password'])

In [ ]:
titles.getResponses()

In [ ]:
df = titles.getDataFrame()
print(df.shape)
df.head()

### Pre-Proccessing

In [ ]:
def userNContent(df, minResponse):
  df2 = df.join(df.groupby(df.user).contentIdx.nunique() > minResponse, on='user',how='left', rsuffix='r')
  return df2.loc[df2.contentIdxr == True, df2.columns.intersection(['user',	'sceneIdx',	'emotion'	,'contentIdx',	'dateAdded'	,'title'	])]


In [ ]:
years = datetime.datetime.now() - datetime.timedelta(days=2*365)
#responses in last 3 years
df= df[df.dateAdded > years]
print(df.shape)
df.head()

In [ ]:
df2 = userNContent(df, 14)
print(df2.shape)
df2.head()
df2.user.nunique()

In [ ]:
km = K_Mean(df2, 5)

### Script Execution

In [ ]:
km.oneVeresusAll(2, n=40, multiplier=3)

In [ ]:
with open('../kmean_result.pkl', 'wb') as handle:
    pickle.dump(km, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
counts = [len(i.users) for i in km.clusters]
cluster = [i.clusterId for i in km.clusters]


In [ ]:
# sns.barplot(x=cluster, y=counts).set_title('Users in Cluster (k=10)')

In [ ]:
km.oneVeresusAll(2, n=40, multiplier=3)

In [ ]:
# with open(f'''kmean_{datetime.now().strftime("%m_%d_%Y")}.pkl''', 'wb') as fid:
#   pickle.dump(km, fid)

In [ ]:
# with open('kmean_05_26_2021.pkl', 'rb') as fid:
#      km = pickle.load(fid)

In [ ]:
# 10000 // 4

##### Calculation

In [ ]:
s1 = Scores(km)

In [ ]:
s1.score(method='standard', n=30, debug=True)

In [ ]:
with open(f'''scores_{datetime.now().strftime("%m_%d_%Y")}.pkl''', 'wb') as fid:
  pickle.dump(s1, fid)

In [ ]:
from datetime import datetime 

In [ ]:
with open(f'''scores_{datetime.now().strftime("%m_%d_%Y")}.pkl''', 'wb') as fid:
  pickle.dump(s1.userScores, fid)

In [ ]:
with open('scores_05_26_2021.pkl', 'rb') as fid:
     s1 = pickle.load(fid)

In [ ]:
sns.histplot(np.array(list(s1.userScores.values()))).set_title(f"Histogram of CAM Scores with n={len(s1.userScores)}")

In [ ]:
from sklearn.preprocessing import minmax_scale, normalize, scale

In [ ]:
# scale data between 0 and 1

scaledValues = minmax_scale(np.array(list(s1.userScores.values())))


sns.histplot(scaledValues).set_title("Historgram with Scaled Scores")

## Genre CLustering

In [ ]:
movies = km.df.contentIdx.unique()
dbPull = Titles(connector)
dbPull.titles = movies
dbPull.getResponses(debug=False)

In [ ]:
%%time
dfs = []
for cont in dbPull.content:
  temp = cont.ratings.loc[:, ['user','rating']]
  temp['id'] = cont.id
  temp['title'] = cont.title
  temp['genres'] = temp.apply(lambda x: cont.genres, axis=1)
  dfs.append(temp)

gdf = pd.concat(dfs)

In [ ]:
gdf.head()

In [ ]:
genres = list(set(itertools.chain.from_iterable(gdf.genres)))
{}

In [ ]:
gdf[gdf.user == np.random.choice(gdf.user.unique())]

In [ ]:
from collections import Counter

In [ ]:
%%time
def getCaScores(user):
  contentList = []
  for hash in s1.users[user].scenes:
    contentList.append(km.df[km.df.hash == hash].iloc[0].contentIdx)


  caScores = {i:0 for i in set(contentList)}
  caCounts = Counter(contentList)

  for idx, content in enumerate(contentList):
    caScores[content] += s1.users[user].scores[idx]

  for ith in caScores:
    caScores[ith] = caScores[ith] / caCounts[ith]
  
  return list(caScores.values())

def getAllSCacores():
  caScores = {}
  for ithUser in s1.users:
    caScores[ithUser] = getCaScores(ithUser)
  
  return caScores



In [ ]:
caScores = getAllSCacores()
caScores

### Payment Modeling

In [ ]:

db = DB(connector['host'], connector['user'],connector['password'])


In [ ]:
counts = pd.DataFrame(db.run(f'''SELECT user_id, COUNT(user_id) 
           FROM results 
           WHERE code = 'EMOJI' 
           GROUP BY user_id ; 
           '''), columns=['user','responses'])

In [ ]:
mediaCounts = pd.DataFrame(db.run(f'''SELECT user_id, COUNT(DISTINCT media_id) 
           FROM results 
           WHERE code = 'EMOJI' 
           GROUP BY user_id'''), columns=['user','mediaCount'])

In [ ]:
earnings = pd.DataFrame(db.run(f'''SELECT user_id, ROUND(SUM(debit)/1000,2), ROUND(SUM(credit)/1000, 2), SUM(credit)
           FROM flow
           WHERE code = 'EMOJI'
           GROUP BY user_id
           ORDER BY SUM(credit) DESC
           '''), columns=['user','redeem','earned','flow']).merge(counts).merge(mediaCounts)
earnings.head()

In [ ]:
user = 1182
print(s1.users[user].score, s1.users[user].cluster)
np.unique(km.users[user].emotions, return_counts=True)

In [ ]:
print(earnings.shape)
nearnings = earnings[(earnings.responses > 10) & (earnings.responses < 50000) ]
nearnings['flow'] = nearnings['flow'].apply(pd.to_numeric)

In [ ]:
sns.regplot( x=nearnings.mediaCount, y=nearnings.flow)

In [ ]:
from sklearn import linear_model


In [ ]:
regr = linear_model.LinearRegression()
_x = nearnings.mediaCount.to_numpy().reshape(-1, 1)
_y =  nearnings.flow.to_numpy().reshape(-1, 1)

In [ ]:
regr.fit(_x, _y)
print('Coefficients: \n', regr.coef_)


### Quality Payments

In [ ]:
def getGamma():
  payouts = []
  for ithUser in s1.users:
    q = max(s1.users[ithUser].scores)
    x_u = km.df[km.df.user == ithUser].contentIdx.unique().shape[0]
    payouts.append(q * x_u)
  ss1 = sum(payouts)

  payouts = []
  for ithUser in s1.users:
    x = km.df[km.df.user == ithUser].contentIdx.unique().shape[0]
    payouts.append(regr.coef_[0][0] * x)
  ss2 = sum(payouts)
  
  return ss2/ss1

def gammaTwo():
  payouts = []
  caScores = caScoreGet()
  for ithUser in s1.users:
    q = max((0, caScores[ithUser]))
    x_u = km.df[km.df.user == ithUser].contentIdx.unique().shape[0]
    payouts.append(q * x_u)
  ss1 = sum(payouts)

  payouts = []
  for ithUser in s1.users:
    x = km.df[km.df.user == ithUser].contentIdx.unique().shape[0]
    payouts.append(regr.coef_[0][0] * x)
  ss2 = sum(payouts)
  

  return ss2/ss1

def caScoreGet():
  scorees = {}
  for user in s1.users:
    scorees[user] = np.mean(s1.users[user].scores)
  return scorees



In [ ]:
gamma = getGamma()
gamma

In [ ]:

def getPayment(user,gammma,beta, alpha, debug=False):
  q_u = s1.userScores[user]
  x_u  = km.df[km.df.user == user].contentIdx.unique().shape[0]
  y_u = regr.predict([[x_u]])[0][0]

  quality = (gamma * q_u * x_u)
  quantity = (x_u * beta)
  if debug:
    print("Score:", q_u)
    print("Quantity: ", quantity)
    print("Quality: ", quality)
  return (alpha * quantity) + ((1-alpha) * quality )

def getPayment2(user,gammma,beta, alpha, caScore, debug=False):
  q_u = caScore[user]
  x_u  = km.df[km.df.user == user].contentIdx.unique().shape[0]
  y_u = regr.predict([[x_u]])[0][0]

  quality = (gamma * q_u * x_u)
  quantity = (x_u * beta)
  if debug:
    print("Score:", q_u)
    print("Quantity: ", quantity)
    print("Quality: ", quality)
  return (alpha * quantity) + ((1-alpha) * quality )

In [ ]:
print(randomUser)
gamma = getGamma()
beta = regr.coef_[0][0]
alpha = .5
print("\n Total Flow Payment", round(getPayment(randomUser, gamma, beta, alpha, debug=True), 2))

In [ ]:
randomUser = np.random.choice(list(s1.users.keys()))
print(randomUser)
gamma = gammaTwo()
caScores = caScoreGet()
beta = regr.coef_[0][0]
alpha = .5
print("\n Total Flow Payment", round(getPayment2(randomUser, gamma, beta, alpha, caScores, debug=True), 2))

In [ ]:
def getAllUser(alpha):
  gamma = getGamma()
  beta = regr.coef_[0][0]
  payment = {}
  for ithUser in s1.users:
    payment[ithUser] = getPayment(ithUser, gamma, beta, alpha, debug=False)
  print(gamma)
  return pd.Series(payment)

def getAllUser2(alpha):
  gamma = gammaTwo()
  caScores = caScoreGet()
  beta = 50
  payment = {}
  for ithUser in s1.users:
    payment[ithUser] = getPayment2(ithUser, gamma, beta, alpha, caScores, debug=False)
  return pd.Series(payment)



In [ ]:
gammaTwo()

In [ ]:
slist = []
for alpha in np.arange(0,1.1,0.2):
  slist.append(getAllUser2(alpha))

In [ ]:
payments = pd.DataFrame(slist).transpose()
payments.head()

In [ ]:
payments.sum()

In [ ]:
payments.sort_values(by=0, ascending=False)

Alpha Level Histograms

In [ ]:
nIters = len(list(payments.columns))
n_cols = 3
n_rows = 2
_, axes = plt.subplots(nrows=n_rows,ncols=n_cols ,figsize=((20,15)))

for i, col in enumerate(payments.columns):
  sns.histplot(payments.loc[:,col], ax=axes[i//n_cols,i%n_cols] ).set_title(f"Alpha = {col * 0.2}")

In [ ]:
payvsaplha = payments.reset_index().rename(columns={'index':'user', 0:'0.0',1:'0.2',2:'0.4',3:'0.6',4:'0.8',5:'1'}).merge(earnings)
payvsaplha['realFlow'] = payvsaplha.apply(lambda x: km.df[km.df.user == x.user].contentIdx.unique().shape[0] * 50, axis=1)
payvsaplha.sort_values(by='flow', ascending=False).head()

In [ ]:
nIters = len(list(payments.columns))
n_cols = 3
n_rows = 2
_, axes = plt.subplots(nrows=n_rows,ncols=n_cols ,figsize=((20,15)))


for i, col in enumerate(['0.0','0.2',	'0.4',	'0.6',	'0.8',	'1']):
  sns.scatterplot(x=payvsaplha['realFlow'], y=payvsaplha[col], ax=axes[i//n_cols,i%n_cols] ).set_title(f'Alphae = {col}')

###### CA score

In [ ]:
ll = {ithuser:np.mean(s1.users[ithuser].scores) for ithuser in s1.users}

In [ ]:
caScores = caScoreGet()

In [ ]:
ca = pd.Series(caScores)
ca.head()

In [ ]:
sns.histplot(caScores)

In [ ]:
bottomCut = list(ca.quantile([.1, .9]))[0]
topCut = list(ca.quantile([.1, .9]))[1]
bottomUsers = np.array(list(ca[ca < bottomCut].index))
topUsers = np.array(list(ca[ca > topCut].index))
print(f"""# Users Avg < {round(bottomCut, 4)}: {bottomUsers.shape[0]} """)
print(f"""# Users Avg > {round(topCut,4)}: {topUsers.shape[0]} """)

Do users with a lower average score respond less (Scenes)?


In [ ]:
bcount = [km.users[i].scenes.shape[0] for i in bottomUsers]
tcount = [km.users[i].scenes.shape[0] for i in topUsers]


print(f'''On average, bottom users repsonded to {round(np.mean(bcount),1)} scenes''')
print(f'''On average, top users repsonded to {round(np.mean(tcount),1)} scenes''')

Do users with a lower average score respond less (Content)?


In [ ]:
bcount = [km.df[km.df.user == i].contentIdx.unique().shape[0] for i in bottomUsers]
tcount = [km.df[km.df.user == i].contentIdx.unique().shape[0] for i in topUsers]

print(f'''On average, bottom users repsonded to {round(np.mean(bcount),1)} content''')
print(f'''On average, top users repsonded to {round(np.mean(tcount),1)} content''')

How does entropy differe between the two groups?

In [ ]:
botEntropy = [km.users[i].entropy for i in bottomUsers]
topEntropy = [km.users[i].entropy for i in topUsers]
normal =  [km.users[i].entropy for i in km.uniqueUsers]

print(f'''The avg entropy for all users is : {round(np.mean(normal),4)} with a STD of {round(np.std(normal), 4)}''')
print(f'''Users with lowest scores have an average entropy of {round(np.mean(botEntropy),4)} with a STD of {round(np.std(botEntropy),4)}''')
print(f'''Users with highest scores have an average entropy of {round(np.mean(topEntropy),4)} with a STD of {round(np.std(topEntropy),4)}''')

How do low scores respond compared to their clusters?

In [ ]:
sceneData = []
randomUser = km.users[np.random.choice(bottomUsers)]
clust = randomUser.getCluster()

for ithU in km.clusters[clust].users:
  __u = km.users[ithU]
  sceneData.append(__u.emotions)


v1,c1 = np.unique(np.hstack(sceneData), return_counts=True)
v2, c2 = np.unique(randomUser.emotions, return_counts=True)

c1norm = c1 / c1.sum()
c2norm = c2/c2.sum()

_, axes = plt.subplots(ncols=2, sharey=True, figsize=(10,5))
axes[0].set_title(f"""Cluster {clust}""")
axes[1].set_title("User")
sns.barplot(y=c1norm, x=v1, order=v1 ,ax=axes[0]) 
sns.barplot(y=c2norm, x=v2, order=v1 , ax=axes[1]) 

**How do high scorers respond compared to their clusters?**

They appear to respond similarly to their cluster as other users -- even low scorers.

In [ ]:
sceneData = []
randomUser = km.users[np.random.choice(topUsers)]
clust = randomUser.getCluster()

for ithU in km.clusters[clust].users:
  __u = km.users[ithU]
  sceneData.append(__u.emotions)


v1,c1 = np.unique(np.hstack(sceneData), return_counts=True)
v2, c2 = np.unique(randomUser.emotions, return_counts=True)

c1norm = c1 / c1.sum()
c2norm = c2/c2.sum()

_, axes = plt.subplots(ncols=2, sharey=True, figsize=(10,5))
axes[0].set_title(f"""Cluster {clust}""")
axes[1].set_title("User")
sns.barplot(y=c1norm, x=v1, order=v1 ,ax=axes[0]) 
sns.barplot(y=c2norm, x=v2, order=v1 , ax=axes[1]) 

## Demographic Analysis

In [ ]:
topUsers

In [ ]:
bottomUsers

In [ ]:
usersDf = db.run(f"""SELECT n.user_id, n.fname, n.lname, n.email, n.sex, n.ethnicity, n.income, n.parent, n.birthdate,a.postal_code, a.city, a.state,n.date_added 
                   FROM users n
                   INNER JOIN user_addresses a ON n.user_id = a.user_id
                   WHERE n.income IS NOT NULL
                   ;""")
usersDf = pd.DataFrame(usersDf, columns=['user','first','last','email','sex','ethnicity','income','parent','dob','zip','city','state','date'])

In [ ]:
usersDf['age'] = usersDf.apply(lambda x:  datetime.now().year - x.dob.year, axis=1)

In [ ]:
usersDf.head()

In [ ]:
eTop1 = usersDf[usersDf.user.isin(topUsers)].ethnicity.value_counts(normalize=True).sort_index(ascending=True)
print(eTop1)
eTop = eTop1.to_numpy()

In [ ]:
eBott1 = usersDf[usersDf.user.isin(bottomUsers)].ethnicity.value_counts(normalize=True).sort_index(ascending=True)
print(eBott1)
eBott = eBott1.to_numpy()

In [ ]:
exp1 = usersDf.ethnicity.value_counts(normalize=True).sort_index(ascending=True)
print(exp1)
exp = exp1.to_numpy()

In [ ]:
from scipy.stats import chisquare

In [ ]:
chisquare(f_obs=eTop, f_exp=exp)

In [ ]:
chisquare(f_obs=eBott, f_exp=exp)

In [ ]:
fig, ax = plt.subplots()
pd.DataFrame([exp1, eTop1]).T.plot(kind='bar',  ax=ax)
ax.legend(["Population", "Top 10%"]);
ax.set_title("High Scoring Demographics VS All Users")

In [ ]:
fig, ax = plt.subplots()
pd.DataFrame([exp1, eBott1]).T.plot(kind='bar',  ax=ax)
ax.legend(["Population", "Bottom 10%"]);
ax.set_title("Low Scoring Demographics VS All Users")

In [ ]:
eTop1 = usersDf[usersDf.user.isin(topUsers)].income.value_counts(normalize=True).sort_index(ascending=True)
print(eTop1)
eTop = eTop1.to_numpy()

In [ ]:
eBott1 = usersDf[usersDf.user.isin(bottomUsers)].income.value_counts(normalize=True).sort_index(ascending=True)
print(eBott1)
eBott = eBott1.to_numpy()

In [ ]:
exp1 = usersDf.income.value_counts(normalize=True).sort_index(ascending=True)
print(exp1)
exp = exp1.to_numpy()

In [ ]:
fig, ax = plt.subplots()
pd.DataFrame([exp1, eTop1]).T.plot(kind='bar',  ax=ax)
ax.legend(["Population", "Top 10%"]);
ax.set_title("High Scorers -- Income VS All Users")

In [ ]:
fig, ax = plt.subplots()
pd.DataFrame([exp1, eBott1]).T.plot(kind='bar',  ax=ax)
ax.legend(["Population", "Bottom 10%"]);
ax.set_title("Low Scoring Demographics VS All Users")

In [ ]:
eTop1 = usersDf[usersDf.user.isin(topUsers)].sex.value_counts(normalize=True).sort_index(ascending=True)
print(eTop1)
eTop = eTop1.to_numpy()

In [ ]:
eBott1 = usersDf[usersDf.user.isin(bottomUsers)].sex.value_counts(normalize=True).sort_index(ascending=True)
print(eBott1)
eBott = eBott1.to_numpy()

In [ ]:
exp1 = usersDf.sex.value_counts(normalize=True).sort_index(ascending=True)
print(exp1)
exp = exp1.to_numpy()

In [ ]:
fig, ax = plt.subplots()
pd.DataFrame([exp1, eTop1]).T.plot(kind='bar',  ax=ax)
ax.legend(["Population", "Top 10%"]);
ax.set_title("High Scorers -- Income VS All Users")

In [ ]:
fig, ax = plt.subplots()
pd.DataFrame([exp1, eBott1]).T.plot(kind='bar',  ax=ax)
ax.legend(["Population", "Bottom 10%"]);
ax.set_title("Low Scoring Demographics VS All Users")

In [ ]:
eTop1 = usersDf[usersDf.user.isin(topUsers)].age
eTop = eTop1.to_numpy()
sns.histplot(eTop).set_title("Age of High Scoring Users")

In [ ]:
eBott1 = usersDf[usersDf.user.isin(bottomUsers)].age
eBott = eBott1.to_numpy()
sns.histplot(eBott).set_title("Age of Low Scoring Users")

In [ ]:
sns.histplot(usersDf.age)

## Length on Platform

In [ ]:
interaction = db.run(f"SELECT user_id, MIN(date_added), MAX(date_added), COUNT(*) FROM flow GROUP BY user_id")

In [ ]:
inDf = pd.DataFrame(interaction, columns=['user','i1','i2', 'nInteractions'])
inDf.head()

In [ ]:
inDf['delta']  = inDf.i2 - inDf.i1

In [ ]:
inDf['deltaday'] = inDf.apply(lambda x: x.delta.days, axis=1)

In [ ]:
active = inDf[inDf.deltaday > 2]

In [ ]:
print('All registered users ', inDf.shape[0])
# all users that have been on platform longer that 1 day
print('Users w/more than one day on platform ',active.shape[0])

In [ ]:
print("Median days on platform: ", active.deltaday.median())

In [ ]:
print(f"Mean: {active.deltaday.describe()['mean']} and STD: {active.deltaday.describe()['std']}")

In [ ]:
print(f"Median of interactions for active users: {active.nInteractions.median()}")

Top and Bottom Users time on platform

In [ ]:
sns.histplot(inDf[inDf.user.isin(topUsers)].deltaday).set_title("Top Users time on Platform")

In [ ]:
sns.histplot(inDf[inDf.user.isin(bottomUsers)].deltaday).set_title("Worst users time on platform")

In [ ]:
oldestLongest =inDf.sort_values(by='nInteractions', ascending=False).iloc[:140]

In [ ]:
topBot = np.intersect1d(oldestLongest.user.to_numpy(), topUsers)
print(topBot.shape[0])

In [ ]:
oldBot = np.intersect1d(oldestLongest.user.to_numpy(), bottomUsers)
print(oldBot.shape[0])

In [ ]:
rid = np.random.choice(topBot)

values, counts = np.unique(km.users[rid].emotions, return_counts=True) 
sns.barplot(x=values, y=counts).set_title(f'User {rid} in cluster {km.users[rid].getCluster()}')

In [ ]:
rid = np.random.choice(oldBot)

values, counts = np.unique(km.users[rid].emotions, return_counts=True) 
sns.barplot(x=values, y=counts).set_title(f'User {rid} in cluster {km.users[rid].getCluster()}')

## Cluster for each user in top or bottom

In [ ]:
values, counts = np.unique(np.array([km.users[i].getCluster() for i in topUsers]), return_counts=True)
sns.barplot(x=values, y=counts).set_title(f'Clusters of Top Users')

In [ ]:
values, counts = np.unique(np.array([km.users[i].getCluster() for i in bottomUsers]), return_counts=True)
sns.barplot(x=values, y=counts).set_title(f'Clusters of Bottom Users')

In [ ]:
values, counts = np.unique(np.array([km.users[i].getCluster() for i in km.uniqueUsers]), return_counts=True)
sns.barplot(x=values, y=counts).set_title(f'Clusters of All Users')

## User Activity

## Clusters

## Clust Viz

In [ ]:
flattened = {}
for user in km.uniqueUsers:
  flattened[user] = np.array([i.vector for i in km.users[user].struct.vectors]).flatten()

In [ ]:
vectors = pd.DataFrame(flattened).T

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(random_state = 42, n_components=2,verbose=1, perplexity=50, n_iter=300, early_exaggeration= 10).fit_transform(vectors)

In [ ]:
vizDf = pd.DataFrame(tsne, columns=['x','y'], index=vectors.index).reset_index().rename(columns={'index':'user'})
vizDf['cluster'] = vizDf.apply(lambda x: km.users[x.user].getCluster(), axis=1)
vizDf.head()

In [ ]:
plt.scatter(vizDf['x'], vizDf['y'], s= 5, c=vizDf['cluster'], cmap='Spectral')
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(7)-0.5).set_ticks(np.arange(6))
plt.title('User Clusters Based Visualization ', fontsize=12);

In [ ]:
import plotly.express as px

In [ ]:
tsne = TSNE(random_state = 42, n_components=3,verbose=1, perplexity=50, n_iter=300, early_exaggeration= 10).fit_transform(vectors)
vizDf = pd.DataFrame(tsne, columns=['x','y','z'], index=vectors.index).reset_index().rename(columns={'index':'user'})
vizDf['cluster'] = vizDf.apply(lambda x: km.users[x.user].getCluster(), axis=1)
vizDf.head()

In [ ]:
fig = px.scatter_3d(
    vizDf, x='x', y='y', z='z',
    color='cluster', hover_data=['user','cluster'])
fig.update_traces(marker_size=8)
fig.show()

In [ ]:
tsne = TSNE(random_state = 42, n_components=2,verbose=1, perplexity=50, n_iter=300, early_exaggeration= 10).fit_transform(vectors)
vizDf = pd.DataFrame(tsne, columns=['x','y'], index=vectors.index).reset_index().rename(columns={'index':'user'})
vizDf['cluster'] = vizDf.apply(lambda x: km.users[x.user].getCluster(), axis=1)
vizDf.head()

In [ ]:
fig = px.scatter(
    vizDf, x='x', y='y',
    color='cluster', hover_data=['user','cluster'])
fig.update_traces(marker_size=8)
fig.show()

In [ ]:
db.run(f"describe user_cluster")

In [ ]:
INSERT INTO table_name (column1, column2, column3, ...)
VALUES (value1, value2, value3, ...);

In [ ]:
for idx, row in vizDf.iterrows():
  if idx % 100 == 0: print(f'[Insert into DB]: Row {idx}')
  print(int(row.cluster), int(row.user))
  db.run(f'''INSERT INTO user_cluster (user_cluster_id, user_id, x, y)
             VALUES ({row.cluster}, {row.user}, {row.x}, {row.y});''')


In [ ]:
import plotly.express as px
import pandas as pd


data = db.run("select * from user_cluster")
df = pd.DataFrame(data, columns=['index','user','cluster','x','y','date'])

fig = px.scatter(
    df, x='x', y='y',
    color='cluster', hover_data=['user','cluster'])
fig.update_traces(marker_size=8)
fig.show()

In [ ]:
db.run("select * from user_cluster")